# Azouaou LOUAIFI   et           Yacine SAHNOUNE


# Stratégie optimale et matrice des gains

In [2]:
from __future__ import annotations
import functools, random
from typing import List, Tuple, Dict
import pulp
V_CACHE: Dict[Tuple[int, int, int], float] = {}
P_CACHE: Dict[Tuple[int, int, int], Tuple[float, ...]] = {}
def get_nash(J):
        lines=range(len(J))
        columns=range(len(J[0]))

        lp = pulp.LpProblem('Nash', pulp.LpMaximize)
        V = pulp.LpVariable('V', cat='Continous')

        a = pulp.LpVariable.dicts("a", (lines),  cat="Continous")
        lp.setObjective(V)

        lp += pulp.lpSum([a[i] for i in lines]) == 1

        for i in lines :
            lp +=a[i]>=0

        for j in columns :
            lp += pulp.lpSum([a[i]*J[i][j] for i in lines])-V>=0


        lp.solve()

        return [pulp.value(lp.objective), [pulp.value(a[i]) for i in lines]]

def sign(u: int) -> int:
    return 1 if u > 0 else -1 if u < 0 else 0

#definir les cas de bases 
def cas_de_base(x: int, y: int, t: int,M: int) -> int | None:
    if t ==  M+1:   return  1
    if t == -(M+1): return -1
    if x == 0 and y == 0:            return sign(t)
    if x == 0:                       return  1 if t > y else -1 if t < y else 0
    if y == 0:
        if x > abs(t) or t > 0:      return  1
        if x < abs(t) and t < 0:     return -1
        return 0
    return None  

#construire la matrice des sous-jeu
def construction_matrice_de_gain(x: int, y: int, t: int, M: int) -> List[List[float]]:
    if cas_de_base(x, y, t,M) is not None:
        raise ValueError("État terminal : pas de matrice.")
    J = [[0.0] * y for _ in range(x)]
    for i in range(1, x + 1):
        for j in range(1, y + 1):
            nx, ny = x - i, y - j
            nt = t + sign(i - j)
            J[i - 1][j - 1] = V_CACHE[(nx, ny, nt)]
    return J

#calculer la valeur de chaque sous jeu
@functools.lru_cache(None)
def calculer_matrice_de_gain(x: int, y: int, t: int, M: int) -> Tuple[float, Tuple[float, ...]]:
    key = (x, y, t)
    # Cas déjà en cache explicite
    if key in V_CACHE:
        return V_CACHE[key], P_CACHE[key]

    # Cas terminal
    g0 = cas_de_base(x, y, t, M)
    if g0 is not None:
        V_CACHE[key] = g0
        P_CACHE[key] = (1.0,)   
        return g0, (1.0,)

    # Construire la matrice des sous-états
    J = [[0.0] * y for _ in range(x)]
    for i in range(1, x + 1):
        for j in range(1, y + 1):
            nx, ny = x - i, y - j
            nt = t + sign(i - j)
            sub_val, _ = calculer_matrice_de_gain(nx, ny, nt, M)
            J[i - 1][j - 1] = sub_val

    val, dist = get_nash(J)
    # Sauvegarde dans les deux caches
    V_CACHE[key] = val
    P_CACHE[key] = tuple(dist)
    return val, tuple(dist)



def valeur_du_jeu(x: int, y: int, t: int,M :int) -> float:
    return calculer_matrice_de_gain(x, y, t, M)[0]


def prob_strategie(x: int, y: int, t: int,M: int) -> List[float]:
    return list(calculer_matrice_de_gain(x, y, t,M)[1])

#choix d'une strategie optimal
def choix_coup(x: int, y: int, t: int,M: int) -> int:
    dist = prob_strategie(x, y, t,M)
    r, cumul = random.random(), 0.0
    for k, p in enumerate(dist, start=1):
        cumul += p
        if r <= cumul:
            return k
    return len(dist)


def execution(x,y,t,c):
    M= c//2
    print(f" La valeur du jeu V({x},{y},{t}) =", valeur_du_jeu(x, y, t,M))
    print("Les proba stratégie :", prob_strategie(x, y, t,M))
    print("Coup tiré :", choix_coup(x, y, t,M))
    

if __name__ == "__main__":    
    x = int(input("Entrez le nombre de pierres restantes pour le joueur 1 (x) : "))
    y = int(input("Entrez le nombre de pierres restantes pour le joueur 2 (y) : "))
    t = int(input("Entrez la position du troll (t) : "))
    c = int(input("Entrez le nombre de cases (c) : "))
    execution(x, y, t, c)
    for row in construction_matrice_de_gain(x, y, t,c//2):
         print(row)

Entrez le nombre de pierres restantes pour le joueur 1 (x) : 5
Entrez le nombre de pierres restantes pour le joueur 2 (y) : 4
Entrez la position du troll (t) : -1
Entrez le nombre de cases (c) : 3
 La valeur du jeu V(5,4,-1) = -0.5
Les proba stratégie : [0.0, 0.5, 0.0, 0.5, 0.0]
Coup tiré : 4
[-0.5, -1, -1, -1]
[0.0, 0.0, -1, -1]
[-1.0, 0.0, 0.0, -1]
[-1.0, -1.0, 0.0, 0]
[-1, -1, -1, 0]


# Simulation 1000 itérations


In [10]:
import random

def random_strategy(x, y, t, M):
    return random.randint(1, y)

#simulation d'une partie 
def simulate_game(x, y, t, c):
    sortir = False
    M = c // 2
    j=0
    while x != 0 and y != 0 and not sortir:       
            # choix des coups
            p1 = choix_coup(x, y, t, M)
            #print("j1:", p1)
            p2 = random_strategy(x, y, t, M)
            #print("j2:", p2)
            # mise à jour des réserves
            x -= p1
            y -= p2

            # déplacement du troll
            if p1 > p2:
                t += 1
                #print("Instance gagnée par Joueur 1")
            else: 
                if p1 < p2:
                    t -= 1
                    #print("Instance gagnée par Joueur 2")
                #else:
                    #print("troll n'a pas bouge")
            if t == -(M+1) or t == M+1:
                    sortir = True
    
    #print("fin de la partie")
    if sortir:
        if t == M+1:
            #print("joueur 1 gagne")
            j=1
        else:
            #print("joueur 2 gagne")
            j=2
    else:
        # épuisement des réserves
        if x == 0 and y == 0:
            if t < 0:
                #print("joueur 1 perd")
                j=2
            #elif t == 0:
                #print("match nul")
            else:
                #print("joueur 2 perd")
                j=1
        elif x == 0:
            if t > y:
                #print("joueur 2 perd")
                j=1
            #elif t == y:
                #print("match nul")               
            else:
                #print("joueur 1 perd")
                j=2
        elif y == 0:
            if abs(t) > x:
                #print("joueur 1 perd")
                j=2
            #elif abs(t) == x:
                #print("match nul")
            else:
                #print("joueur 2 perd")
                j=1
    return j
                
    
#simulation avec 1000 fois
if __name__ == "__main__":
    j1 = 0
    j2 = 0
    mn = 0
    x = int(input("Entrez le nombre de pierres restantes pour le joueur 1 (x) : "))
    y = int(input("Entrez le nombre de pierres restantes pour le joueur 2 (y) : "))
    t = int(input("Entrez la position du troll (t) : "))
    c = int(input("Entrez le nombre de cases (c) : "))
    for i in range(1000):
        j = simulate_game(x, y, t, c)
        if j == 0:
            mn += 1
        elif j == 1:
            j1 += 1
        else:
            j2 += 1

    print(f"Joueur 1 gagne : {j1} fois")
    print(f"Joueur 2 gagne : {j2} fois")
    print(f"Matchs nuls   : {mn} fois")



Entrez le nombre de pierres restantes pour le joueur 1 (x) : 30
Entrez le nombre de pierres restantes pour le joueur 2 (y) : 30
Entrez la position du troll (t) : 0
Entrez le nombre de cases (c) : 15
Joueur 1 gagne : 1000 fois
Joueur 2 gagne : 0 fois
Matchs nuls   : 0 fois
